# 🌳 Dendritic YOLOv8 - Object Detection with Dendritic Optimization

This notebook demonstrates integrating PerforatedAI's dendritic optimization into YOLOv8n for object detection.

**Hackathon Project**: Dendritic Optimization Challenge

**Goal**: Improve YOLOv8 performance or reduce model size using artificial dendrites

**Hardware**: Optimized for Google Colab Free Tier (T4 GPU)

## 📦 Setup and Installation

In [ ]:
# Clone PerforatedAI repository
!git clone https://github.com/PerforatedAI/PerforatedAI.git
%cd PerforatedAI

# Install PerforatedAI
!pip install -q setuptools
!pip install -q -e .

# Install YOLOv8 and other dependencies
!pip install -q ultralytics wandb

print("✅ Installation complete!")

In [ ]:
# Verify GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 🔑 Weights & Biases Setup (Optional)

For experiment tracking and hyperparameter sweeps

In [ ]:
import wandb

# Login to W&B (get API key from https://wandb.ai/authorize)
wandb.login()

# Or set API key directly:
# import os
# os.environ['WANDB_API_KEY'] = 'your-api-key-here'

## 📥 Download Dataset

We'll use COCO128 for fast iteration (128 images from COCO)

In [ ]:
from ultralytics import YOLO

# Download COCO128 dataset (will be auto-downloaded on first run)
model = YOLO('yolov8n.pt')
print("✅ YOLOv8n model downloaded!")
print("✅ COCO128 dataset will auto-download during training")

## 🎯 Baseline Training (No Dendrites)

First, let's train a baseline YOLOv8n model for comparison

In [ ]:
# Train baseline model
baseline_model = YOLO('yolov8n.pt')

baseline_results = baseline_model.train(
    data='coco128.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    project='yolov8-baseline',
    name='baseline',
    patience=20,
    save=True,
    plots=True,
    verbose=True,
)

# Validate baseline
baseline_metrics = baseline_model.val()
print(f"\n{'='*50}")
print(f"Baseline Results:")
print(f"mAP@0.5: {baseline_metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {baseline_metrics.box.map:.4f}")
print(f"{'='*50}")

## 🌳 Dendritic Training

Now let's train with dendritic optimization!

In [ ]:
%cd /content/PerforatedAI/Examples/hackathonProjects/dendritic-yolov8

# Download training script if not present
import os
if not os.path.exists('train_yolov8_dendritic.py'):
    !wget https://raw.githubusercontent.com/PerforatedAI/PerforatedAI/main/Examples/hackathonProjects/dendritic-yolov8/train_yolov8_dendritic.py

# Run dendritic training
!python train_yolov8_dendritic.py \
    --data coco128.yaml \
    --epochs 50 \
    --imgsz 640 \
    --batch 16 \
    --project dendritic-yolov8 \
    --name dendritic-run \
    --patience 20

## 🔬 W&B Hyperparameter Sweep (Optional)

Run comprehensive experiments with different dendritic configurations

In [ ]:
# Initialize W&B sweep
!python train_yolov8_dendritic.py \
    --use-wandb \
    --data coco128.yaml \
    --epochs 50 \
    --batch 16 \
    --count 10

## 📊 View Results

Check the PAI folder for automatically generated graphs

In [ ]:
from IPython.display import Image, display
import os

# Display PAI graph (required for hackathon submission)
if os.path.exists('PAI/PAI.png'):
    print("🌳 Dendritic Optimization Results:")
    display(Image('PAI/PAI.png'))
else:
    print("⚠️ PAI graph not found. Make sure training completed.")

# Display training results
if os.path.exists('dendritic-yolov8/dendritic-run/results.png'):
    print("\n📈 Training Curves:")
    display(Image('dendritic-yolov8/dendritic-run/results.png'))

# Display confusion matrix
if os.path.exists('dendritic-yolov8/dendritic-run/confusion_matrix.png'):
    print("\n🎯 Confusion Matrix:")
    display(Image('dendritic-yolov8/dendritic-run/confusion_matrix.png'))

## 📈 Compare Results

In [ ]:
import pandas as pd

# Load results from CSV if available
if os.path.exists('PAI/PAIbest_test_scores.csv'):
    results_df = pd.read_csv('PAI/PAIbest_test_scores.csv')
    print("\n🔬 Dendritic Optimization Results:")
    print(results_df)
    
    # Calculate improvement
    if len(results_df) > 1:
        baseline_acc = results_df.iloc[0]['Validation Score']
        dendritic_acc = results_df.iloc[-1]['Validation Score']
        
        baseline_params = results_df.iloc[0]['Parameters']
        dendritic_params = results_df.iloc[-1]['Parameters']
        
        print(f"\n{'='*60}")
        print(f"📊 FINAL RESULTS")
        print(f"{'='*60}")
        print(f"Baseline Accuracy: {baseline_acc:.2f}%")
        print(f"Dendritic Accuracy: {dendritic_acc:.2f}%")
        print(f"Improvement: {dendritic_acc - baseline_acc:.2f} percentage points")
        print(f"\nBaseline Parameters: {baseline_params:,}")
        print(f"Dendritic Parameters: {dendritic_params:,}")
        print(f"Parameter Change: {((dendritic_params - baseline_params) / baseline_params * 100):.2f}%")
        
        # Calculate Remaining Error Reduction
        baseline_error = 100 - baseline_acc
        dendritic_error = 100 - dendritic_acc
        if baseline_error > 0:
            rer = ((baseline_error - dendritic_error) / baseline_error) * 100
            print(f"\n🎯 Remaining Error Reduction: {rer:.1f}%")
        print(f"{'='*60}")
else:
    print("⚠️ Results CSV not found")

## 💾 Download Results

Download all results for your hackathon submission

In [ ]:
# Create a zip file with all results
!zip -r dendritic_yolov8_results.zip \
    PAI/ \
    dendritic-yolov8/ \
    yolov8-baseline/ \
    -x "*.pt" "*weights*"

print("\n✅ Results zipped!")
print("Download 'dendritic_yolov8_results.zip' from the Files panel on the left.")

from google.colab import files
files.download('dendritic_yolov8_results.zip')

## 🚀 Next Steps

1. **Review PAI/PAI.png** - This is required for hackathon submission
2. **Check PAI/PAIbest_test_scores.csv** - Contains detailed metrics
3. **Compare baseline vs dendritic** - Calculate Remaining Error Reduction
4. **Create your case study** - Follow the hackathon README format
5. **Submit PR** - Add your project to Examples/hackathonProjects/

## 📚 Resources

- [PerforatedAI Docs](https://github.com/PerforatedAI/PerforatedAI)
- [Hackathon Submission Format](https://github.com/PerforatedAI/PerforatedAI/tree/main/Examples/hackathonProjects/mnist-example-submission)
- [YOLOv8 Docs](https://docs.ultralytics.com/)
- [Weights & Biases](https://wandb.ai/)

## 🎉 Good Luck!

Questions? Check the [PerforatedAI Discord](https://discord.gg/Fgw3FG3Hzt)